<a href="http://folk.ntnu.no/leifh/teaching/tkt4140/._main057.html" target="_blank">Iterative Methods</a>

N = 4 <br>
prototype

<h4>Item <i>"a)"</i> Onde f = 0 e u(x,y)= alpha * np.exp(x) * np.sin(y) na fronteira</h4>

In [45]:
import numpy as np
import pandas as pd
from time import time

<h3>Funções Auxiliares</h3>

<h4>Jacobi</h4>

In [46]:
def jacobi_method(U_old, U_new, h, F):
    for j in range(1,N):
        for i in range(1,N):
            U_new[i, j] = 0.25*(U_old[i-1, j] + U_old[i+1, j] + U_old[i, j-1] + U_old[i, j+1] + h**2 * F[i, j])   

<h4>Gauss-Seidel</h4>

In [47]:
def g_s_method(U_old, U_new, h, F):
    for j in range(1,N):
        for i in range(1,N):
            U_new[i, j] = 0.25*(U_new[i-1, j] + U_old[i+1, j] + U_new[i, j-1] + U_old[i, j+1] + h**2 * F[i, j])   

<h4>SOR</h4>

In [48]:
def sor_method(w, U_old, U_new, h, F):
    for j in range(1,N):
        for i in range(1,N):
            U_new[i,j] =  (w/4.0) * (U_new[i-1,j] + U_old[i+1,j] + U_new[i,j-1] + U_old[i,j+1] + h**2 *F[i,j]) + (1.-w)*U_old[i,j];


<h4>PASSO 1 e 2</h4>

In [49]:
def initialize_matrixes():
    U_old = np.zeros((N+1,N+1))
    U_new = np.zeros((N+1,N+1))
    F = np.zeros((N+1,N+1)) 
    
    return U_old, U_new, F

<h4>PASSO 3</h4>

In [50]:
def initialize_F_2A(F):
    
    for j_y in range(N+1):
        for i_x in range(N+1):
            F[i_x, j_y] = -100./8.84
    
    return F

In [64]:
def initialize_F_2B(F):
    
    for j_y in range(N+1):
        for i_x in range(N+1):
            F[i_x, j_y] = -10./75. * np.sin(np.pi * (i_x*h + j_y*h)) * 10**4
    
    return F

<h4>PASSO 4</h4>

In [52]:
def update_border(U_old, U_new):
    # Parte inferior e superior da Malha
    for i in range(N+1):
        U_old[i, 0] = 0.0
        U_old[i, N] = 110.

    # Parte esquerda e direita da Malha
    for j in range(N+1):
        U_old[0, j] = 110. * np.sin((np.pi/2.)*(h*j))
        U_old[N, j] = 110. * np.sin((np.pi/2.)*(h*j))    
    
    #U_old e U_new têm o mesmo valor nas bordas
    U_new = U_old.copy()
    
    return U_new, U_old

In [53]:
def calculate_difference(U_new, U_old, N):
    
    quadratic_diff = (U_new[1:N-1, 1:N-1] - U_old[1:N-1, 1:N-1]) **2
    return h * np.sqrt((quadratic_diff.sum()))

<h4>PASSO 5-7</h4>

In [54]:
def call_g_s(U_old, U_new, h, F, TOL, MAXITER):
    
    start = time()
    n_iterations = 0
    
    for it in range(80000):

        g_s_method(U_old, U_new, h, F)

        if calculate_difference(U_new, U_old, N) <= TOL:
            print('Gauss-Seidel convergiu com %d iterações' %(it))
            n_iterations = it
            break

        U_old = U_new.copy()

    end = time()
    return ['Gauss-Seidel', N, n_iterations, (end-start)]


In [55]:
def call_jacobi(U_old, U_new, h, F, TOL, MAXITER):
    
    start = time()
    n_iterations = 0

    for it in range(80000):
        jacobi_method(U_old, U_new, h, F)

        if calculate_difference(U_new, U_old, N) <= TOL:
            print('Jacobi convergiu com %d iterações' %(it))
            n_iterations = it
            break

        U_old = U_new.copy()

    end = time()  
    return ['Jacobi', N, n_iterations, (end-start)]


In [56]:
def call_sor(U_old, U_new, h, F, TOL, MAXITER):
    
    start = time()
    n_iterations = 0
    w = 2./(1. + np.sin(np.pi*h))
    
    for it in range(80000):
        sor_method(w, U_old, U_new, h, F)

        if calculate_difference(U_new, U_old, N) <= TOL:
            print('SOR convergiu com %d iterações' %(it))
            n_iterations = it
            break

        U_old = U_new.copy()

    end = time()    
    return ['SOR', N, n_iterations, (end-start)]


In [57]:
df = pd.DataFrame({},columns=['Método', 'tamanho de N', 'num. iterações', 'tempo(seg)'])

In [62]:
N=2**6
MAXITER=80000

alpha = 3.0
h=1./N

TOL = 10**-5 * h

U_old, U_new, F = initialize_matrixes()
F = initialize_F_2A(F)
U_new, U_old = update_border(U_old, U_new)

call_jacobi(U_old, U_new, h, F, TOL, MAXITER)
# call_g_s(U_old, U_new, h, F, TOL, U_answer, MAXITER)
# call_sor(U_old, U_new, h, F, TOL, U_answer, MAXITER)

#df.loc[-1] = call_jacobi(U_old, U_new, h, F, TOL, U_answer, MAXITER)
#df.index += 1

Jacobi convergiu com 10697 iterações


['Jacobi', 64, 10697, 74.34491753578186]

In [63]:
N=2**6
MAXITER=80000

alpha = 3.0
h=1./N

TOL = 10**-5 * h

U_old, U_new, F = initialize_matrixes()
F = initialize_F_2B(F)
U_new, U_old = update_border(U_old, U_new)

call_jacobi(U_old, U_new, h, F, TOL, MAXITER)
# call_g_s(U_old, U_new, h, F, TOL, U_answer, MAXITER)
# call_sor(U_old, U_new, h, F, TOL, U_answer, MAXITER)

#df.loc[-1] = call_jacobi(U_old, U_new, h, F, TOL, U_answer, MAXITER)
#df.index += 1

Jacobi convergiu com 10704 iterações


['Jacobi', 64, 10704, 73.83471608161926]

In [65]:
N=2**6
MAXITER=80000

alpha = 3.0
h=1./N

TOL = 10**-5 * h

U_old, U_new, F = initialize_matrixes()
F = initialize_F_2B(F)
U_new, U_old = update_border(U_old, U_new)

call_jacobi(U_old, U_new, h, F, TOL, MAXITER)
# call_g_s(U_old, U_new, h, F, TOL, U_answer, MAXITER)
# call_sor(U_old, U_new, h, F, TOL, U_answer, MAXITER)

#df.loc[-1] = call_jacobi(U_old, U_new, h, F, TOL, U_answer, MAXITER)
#df.index += 1

Jacobi convergiu com 10704 iterações


['Jacobi', 64, 10704, 74.41035056114197]

In [ ]:
df